In [ ]:
from datetime import datetime, timezone
from typing import Dict, List, Optional, Tuple
from pyspark.sql import DataFrame
from delta.tables import DeltaTable
from pyspark.sql.types import StructType, StructField, TimestampType, StringType
import logging
import pyspark.sql.functions as F

CATALOG = ""
VOLUME_CATALOG = "main"
VOLUME_SCHEMA = "engenharia_dados"
VOLUME_NAME = "aviacao_landing"
LANDING_CSV_BASE_PATH = f"/Volumes/{VOLUME_CATALOG}/{VOLUME_SCHEMA}/{VOLUME_NAME}/aviacao/landing"
BRONZE_SCHEMA = "aviacao_bronze"
META_SCHEMA = "aviacao_meta"
ORIGEM_SISTEMA = "postgres-aviacao"

TABLE_CONFIGS: Dict[str, Dict] = {
    "companhias_aereas": {"schema": "aviacao", "business_key": ["id"]},
    "modelos_avioes": {"schema": "aviacao", "business_key": ["id"]},
    "aeroportos": {"schema": "aviacao", "business_key": ["id"]},
    "aeronaves": {"schema": "aviacao", "business_key": ["id"]},
    "funcionarios": {"schema": "aviacao", "business_key": ["id"]},
    "clientes": {"schema": "aviacao", "business_key": ["id"]},
    "voos": {"schema": "aviacao", "business_key": ["id"]},
    "reservas": {"schema": "aviacao", "business_key": ["id"]},
    "bilhetes": {"schema": "aviacao", "business_key": ["id"]},
    "bagagens": {"schema": "aviacao", "business_key": ["id"]},
    "manutencoes": {"schema": "aviacao", "business_key": ["id"]},
    "tripulacao_voo": {"schema": "aviacao", "business_key": ["id"]},
}

TABLE_SCHEMAS: Dict[str, StructType] = {}

logger = logging.getLogger("aviacao_bronze")
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(name)s - %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)
logger.setLevel(logging.INFO)


# Funções de criação e verificação de esquemas e tabelas
def qname(schema: str, table: str) -> str:
    if CATALOG:
        return f"{CATALOG}.{schema}.{table}"
    return f"{schema}.{table}"


def now_utc():
    return datetime.now(timezone.utc)


def init_schema(schema_name: str) -> None:
    schema_qualified = f"{CATALOG}.{schema_name}" if CATALOG else schema_name
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_qualified}")

def init_watermark_table() -> None:
    init_schema(META_SCHEMA)
    wm_table = qname(META_SCHEMA, "silver_bronze_watermark")
    print(f"[GLOBAL] Garantindo existência da tabela de watermark: {wm_table}")
    
    # Criação da tabela se não existir
    spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {wm_table} (
            tabela STRING NOT NULL,
            ultima_data_ref TIMESTAMP NOT NULL,
            ultima_execucao_ts TIMESTAMP NOT NULL
        )
        USING DELTA
    """)
    
    # A verificação para a constraint pk_watermark_incremental
    # Caso a constraint já exista, não vamos tentar criá-la novamente.
    try:
        spark.sql(f"""
            ALTER TABLE {wm_table} ADD CONSTRAINT pk_watermark_incremental PRIMARY KEY (tabela)
        """)
    except Exception as e:
        print(f"[INFO] Constraint pk_watermark_incremental já existe em {wm_table}, ignorando criação.")

def ensure_schema_exists(schema_name: str) -> None:
    try:
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
        print(f"[INFO] Schema {schema_name} criado ou já existente.")
    except Exception as e:
        print(f"[ERROR] Erro ao criar o schema {schema_name}: {str(e)}")


def ensure_delta_table_exists(table_name: str) -> None:
    # Garantir que o schema exista antes de criar a tabela
    ensure_schema_exists(SILVER_SCHEMA)
    
    silver_table = qname(SILVER_SCHEMA, table_name)
    
    try:
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS {silver_table} (
                id BIGINT,
                nome STRING,
                codigo_iata STRING,
                codigo_icao STRING,
                pais STRING,
                alianca STRING,
                ativo BOOLEAN,
                vigencia_inicio TIMESTAMP,
                vigencia_fim TIMESTAMP,
                is_current BOOLEAN,
                origem_sistema STRING,
                bronze_load_ts TIMESTAMP,
                aud_dh_criacao TIMESTAMP,
                aud_dh_alteracao TIMESTAMP,
                attr_hash STRING
            )
            USING DELTA
        """)
        print(f"[INFO] Tabela Delta {silver_table} criada ou já existente.")
    except Exception as e:
        print(f"[ERROR] Erro ao criar a tabela Delta {silver_table}: {str(e)}")


# Função para ler dados incrementais da tabela Bronze
def read_bronze_incremental(table_name: str, last_processed_ts: str) -> DataFrame:
    # Alterando para usar o nome correto da tabela de bronze com o sufixo "_changelog"
    bronze_table = qname(BRONZE_SCHEMA, f"{table_name}_changelog")

    # Verifica se a tabela existe
    if not spark.catalog.tableExists(bronze_table):
        raise ValueError(f"A tabela {bronze_table} não existe.")
    
    # Lê os dados da tabela de Bronze e filtra com o timestamp
    df_bronze = spark.table(bronze_table).filter(F.col("bronze_load_ts") > last_processed_ts)
    
    return df_bronze


# Função para processar dimensões usando SCD Tipo 2
def process_scd2_dimension(table_name: str) -> None:
    """
    Função para processar dimensões usando SCD Tipo 2.
    """
    print(f"================ INÍCIO PROCESSAMENTO SCD2 DIMENSÃO: {table_name} ================")

    # Obtendo o timestamp do último processamento
    last_processed_ts = get_last_processed_ts(table_name)

    # Lendo os dados incrementais da tabela de bronze
    df_bronze = read_bronze_incremental(table_name, last_processed_ts)

    df_bronze.show(10)
    df_bronze.printSchema()
    print(f"================ FIM DA LEITURA DA DF_BRONZE ================")

    # Garantindo que a tabela Delta de destino (Silver) existe
    ensure_delta_table_exists("dim_companhias_aereas")

    # Definindo a janela para classificação dos registros
    w = Window.partitionBy("id").orderBy(F.col("data_ref").desc(), F.col("bronze_load_ts").desc())

    # Criando as mudanças para comparação com a tabela Silver
    df_changes = (
        df_bronze
        .withColumn("row_number", F.row_number().over(w))
        .filter(F.col("row_number") == 1)
        .drop("row_number")
    )

    # Definindo as colunas para gerar o hash de atributo (para identificar mudanças)
    cols_to_hash = ["nome", "codigo_iata", "codigo_icao", "pais", "alianca", "ativo"]
    df_changes = df_changes.withColumn(
        "attr_hash",
        F.sha2(F.concat_ws("||", *[F.col(c).cast("string") for c in cols_to_hash]), 256)
    )

    df_changes.show(10)
    print(f"================ FIM DA LEITURA DA DF_CHANGES ================")

    # Verifique se df_changes está vazio
    if df_changes.count() == 0:
        print(f"[INFO] Nenhuma mudança encontrada para {table_name}. A tabela Bronze está sem atualizações.")
    else:
        print(f"[INFO] Encontradas {df_changes.count()} mudanças para {table_name}.")
    
    # Carregando a tabela Silver (já existente)
    silver_table = qname(SILVER_SCHEMA, "dim_companhias_aereas")
    df_silver = spark.table(silver_table).filter(F.col("is_current") == True)

    # Realizando a junção entre as mudanças e a tabela Silver
    df_join = df_changes.alias("chg").join(
        df_silver.alias("dim"),
        on=[F.col("chg.id") == F.col("dim.cia_id")],
        how="left"
    )

    # Preparando os dados para mesclar na tabela Silver
    df_to_merge = (
        df_join
        .filter(
            (F.col("dim.cia_id").isNull()) |
            (F.col("dim.attr_hash") != F.col("chg.attr_hash"))
        )
        .select(
            "chg.id",  # Usando 'id' da tabela de mudanças
            "chg.nome",
            "chg.codigo_iata",
            "chg.codigo_icao",
            "chg.pais",
            "chg.alianca",
            "chg.ativo",
            "chg.data_ref",
            "chg.bronze_load_ts",
            "chg.origem_sistema",
            "chg.attr_hash"
        )
    )

    df_to_merge.show()

    # Realizando o MERGE na tabela Delta Silver
    delta_dim = DeltaTable.forName(spark, silver_table)

    delta_dim.alias("dim").merge(
        df_to_merge.alias("chg"),
        "CAST(dim.cia_id AS BIGINT) = CAST(chg.id AS BIGINT) AND dim.is_current = true"
    ) \
    .whenMatchedUpdate(
        condition="dim.attr_hash <> chg.attr_hash",
        set={
            "vigencia_fim": "chg.data_ref - INTERVAL 1 MICROSECOND",
            "is_current": "false",
            "aud_dh_alteracao": "current_timestamp()"
        }
    ) \
    .whenNotMatchedInsert(
        values={
            "cia_id": "chg.id",
            "nome": "chg.nome",
            "codigo_iata": "chg.codigo_iata",
            "codigo_icao": "chg.codigo_icao",
            "pais": "chg.pais",
            "alianca": "chg.alianca",
            "ativo": "chg.ativo",
            "vigencia_inicio": "chg.data_ref",
            "vigencia_fim": "TIMESTAMP '9999-12-31 23:59:59'",
            "is_current": "true",
            "origem_sistema": "chg.origem_sistema",
            "bronze_load_ts": "chg.bronze_load_ts",
            "aud_dh_criacao": "current_timestamp()",
            "aud_dh_alteracao": "current_timestamp()",
            "attr_hash": "chg.attr_hash",
        }
    ) \
    .execute()

    spark.sql("SELECT * FROM aviacao_silver.dim_companhias_aereas WHERE is_current = TRUE").show()

    print(f"================ FIM PROCESSAMENTO SCD2 DIMENSÃO: {table_name} ================")


def main() -> None:
    print("Iniciando o processamento da camada Silver...")
    init_watermark_table()
    process_scd2_dimension("companhias_aereas")

if __name__ == "__main__":
    main()
